In [ ]:
import pandas as pd
import json
import glob
import ast
from pandas.io.json import json_normalize

pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = False


In [ ]:
def data_reader(path_regex):
    datas = {}
    for filename in glob.iglob(path_regex,recursive = True):
        with open(filename, 'r') as f:
            try:
                datas[filename] = json.load(f)
            except:
                print(filename)
                print('oops')
    return datas
    
datas = data_reader('./**/*.json')
processes = []
for k, v in datas.items():
    if type(v) is not list:
        for k2,v2 in v.items():
            if k2 == "_source":
                v2['process']['eventname'] = k.split('/')[-1]
                processes.append(v2['process'])
    elif type(v) is list:
        for x in v:
            try:
                x['_source']['process']['eventname'] = k.split('/')[-1]
                processes.append(x['_source']['process'])
            except:
                print(k)
                print('oops')
    else:
        print(k)

In [ ]:
## This does not handle all possible cases. I tried to clean up the events, but the json isn't nested consistently across the files. 
## This currently misreads files such as macOS-ATTACK-DATASET/Persistence/persistence_emond_rules_filemod.json where there are multiple events.

argument = input()
event_names = []
command_lines = []
for proc in processes:
    if proc.get('args', None) != None and argument in proc['args']:
        event_names.append(proc['eventname'])
        command_lines.append(proc['command_line'])
    else:
        pass

data = {'event_names': event_names, 'command_lines': command_lines}
df = pd.DataFrame(data=data)

df